In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import h5py

### Setup

In [2]:
os.chdir('..')
os.getcwd()

'c:\\Users\\delgr\\Projects\\advancedPython\\Day6a'

### Set Script Parameters

In [3]:
session_dir = './data/extracted/20161214_cori_steinmetz'

In [4]:
session_path = Path(session_dir)
assert session_path.exists()

### Load Data

In [14]:
trials = pd.DataFrame(np.load(session_path / 'trials/trials.npy'),)
trials.set_index('trial', inplace=True)
trials

,contrast_left,contrast_right,stim_onset,licks,response,feedback,gocue,response_time
trial,,,,,,,,
1,100,0,500,11,b'right',b'pos',1027.0,1610.0
2,0,50,3000,12,b'left',b'pos',3374.0,3070.0
3,100,50,5500,13,b'right',b'pos',5825.0,6170.0
4,0,0,8000,0,b'neutral',b'pos',8261.0,10130.0
5,50,100,10500,7,b'right',b'neg',10662.0,NaN
...,...,...,...,...,...,...,...,...
360,50,25,898000,0,b'None',b'None',NaN,NaN
361,50,25,900500,0,b'None',b'None',NaN,NaN
362,0,50,903000,0,b'None',b'None',NaN,NaN


In [16]:
trials_da = xr.Dataset.from_dataframe(trials)
trials_da

<xarray.Dataset> Size: 16kB
Dimensions:         (trial: 364)
Coordinates:
  * trial           (trial) uint16 728B 1 2 3 4 5 6 ... 359 360 361 362 363 364
Data variables:
    contrast_left   (trial) uint8 364B 100 0 100 0 50 0 0 ... 100 50 50 0 25 100
    contrast_right  (trial) uint8 364B 0 50 50 0 100 0 0 ... 100 25 25 50 0 100
    stim_onset      (trial) int64 3kB 500 3000 5500 ... 903000 905500 908000
    licks           (trial) uint8 364B 11 12 13 0 7 3 0 0 0 ... 0 0 0 0 0 0 0 0
    response        (trial) object 3kB b'right' b'left' ... b'None' b'None'
    feedback        (trial) object 3kB b'pos' b'pos' b'pos' ... b'None' b'None'
    gocue           (trial) float64 3kB 1.027e+03 3.374e+03 ... nan nan
    response_time   (trial) float64 3kB 1.61e+03 3.07e+03 6.17e+03 ... nan nan

### Wheel Movement

In [ ]:
session_path.

WindowsPath('data/extracted/20161214_cori_steinmetz')